# RNN原理
![](imgs/24_human_brain.png)
图中的逻辑并不是在说完“我来找你玩游”之后进入大脑来处理的，而是每个字都在脑子里进行着处理，将图9-1中的每个字分别裁开，在语言模型中就形成了一个循环神经网络，图9-1中的逻辑可以用下面的伪码表示:
___
input我+ empty—input）→output我
（input来+ output我）→output来
（input找+ output来）→output找
（input你+ output找）→output你
……
___
即，每个预测的结果都会放到下一个输入里进行运算，与下一次的输入一起来生成下一次的结果。![](imgs/24_rnn_structure.png)上图可以看成是一个链式结构。我们的大脑并不是简单的存储，而是链式的、有顺序的存储。

这种“链式的、有顺序存储”很节省空间，对于中间状态的序列，我们的大脑没有选择直接记住，而是存储计算方法。当我们需要取值时，直接将具体的数据输入。

# RNN网络的应用领域
对于序列化的特征的任务，都适合采用RNN网络来解决。细分起来可以**有情感分析（Sentiment Analysis）、关键字提取（Key Term Extraction）、语音识别（Speech Recognition）、机器翻译（Machine Translation）和股票分析**等。

# 正向传播过程
![](imgs/24_rnn_forward-propagation.png)RNN结构如左侧图，A代表网络，xt代表时刻输入的x,ht代表网络生成的结果，A中间又画出了一条线指向自己，是表明上一时刻的输出接着输入到了A里面。

当有一系列的x输入到图左侧结构中，展开就变成了右侧的样子，其实就是一个含有隐藏层的网络，只不过隐藏层的输出变成了两份，一份传到下一个节点，另一个传给本身节点。时序图如下：![](imgs/24_Timing_diagram.png)
假设有3个时序t1,t2,t3，如上图所示，在RNN中可以分解成以下三个步骤：
1. 开始时t1通过自己的输入权重和0作为输入，生成了out1。
2. out1通过自己的权重生成了h1,然后和t2经过输入权重转化后一起作为输入，生成了out2。
3. out2通过同样的隐藏层权重生成了h2,然后和t3经过输入权重转化后一起作为输入，生成了out2。

# 随时间反向传播
与单神经元相似，RNN也需要反向传播误差来调整自己的参数。RNN网络使用随时间反向传播（BackPropagation Through Time，BPTT）的链式求导算法来反向传播误差。![](imgs/24_BP_backpropagation.png)
这是一个含有一个隐藏层的网络结构。隐藏层只有一个节点。具体的过程如下：

（1）有一个批次含有3个数据A、B、C，批次中每个样本有两个数（x1、x2）通过权重（w1、w2）来到隐藏层H并生成批次h，如图9-5中w1、w2两条直线所在方向。
（2）该批次的h通过隐藏层权重p1生成最终的输出结果y。

（3）y与最终的标签p比较，生成输出层误差less（y，p）。

（4）less（y，p）与生成y的导数相乘，得到Del_y。Del_y为输出层所需要的修改值。

（5）将h的转置与del_y相乘得到del_p1。这是源于h与p1相等得到的y，见第（2）步。

（6）最终将该批次的del_p1求和并更新到p1上。

（7）同理，再将误差反向传递到上一层：计算Del_h。得到Del_h后再计算del_w1、del_w2并更新。

## BPTT算法：
![](imgs/24_bptt.png)
同样是一个批次的数据A、B、C，按顺序进入循环神经网络。正向传播的实例是B正在进入神经网络的过程，可以看到A的h参与进来并一起经过P1生成了B的y。因为C还没有进入，为了清晰，这里用灰色表示。

当所有块都进入之后，会将p标签与输出进行Del_y的运算，由于C块的y是最后生成的，所以我们先从C块开始对h的输出传递误差Del_h。

图中的反向传播是表示C块已经反向传播完成，开始B块反向传播的状态，可以看到B块Del_h是由B块的del_y和C块的del_h通过计算得来的。这就是与BP算法不同的地方（BP中Del_h直接与自己的Del_y相关，不会与其他的值有联系）。

作为一个批次的数据，正向传播时是沿着ABC的顺序，当反向传播时，就按照正向传播的相反顺序，即每个节点的CBA挨个计算并传递梯度。